In [2]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from bs4 import BeautifulSoup
import pickle

# Functions Scrape Data and Get Info Df's For SkinStore

### *****Need to figure out how to get images in full df

In [3]:
def get_products_by_type(product_type,maxpages):
    types=[]
    for i in range(0,maxpages):
        response=requests.get('https://www.skinstore.com/skin-care/{}.list?pageNumber={}'.format(product_type,i))
        soup=BeautifulSoup(response.content,'html.parser')
        prods=soup.findAll('h2',{'class':'productBlock_productName'})
        for p in prods:
            types.append(p.text.strip('\n'))
    return types

def get_products_by_problem(problem,maxpages):
    dfdict=[]
    prodlist=[]
    for i in range(0,maxpages):
        response=requests.get('https://www.skinstore.com/skin-care/skincare-concern/{}.list?pageNumber={}'.format(problem,i))
        soup=BeautifulSoup(response.content,'html.parser')
        brands=soup.findAll('h2',{'class':'productBlock_productName'})
        ratings=soup.findAll('span',{'class':'productBlock_ratingValue'})
        urls=soup.findAll('span',{'class':'js-enhanced-ecommerce-data hidden'},{'data-product-id':True})
        prices=soup.findAll('span',{'class':'productBlock_priceValue'})
#         images=soup.findAll('div',{'class':'productBlock_imageContainer'},{'src':True})
        for b,r,p,u in zip(brands,ratings,prices,urls):
            prodlist.append(b.text.strip('\n'))
            dfdict.append({'prodName':b.text.strip('\n'),'rating':r.text,'price':p.text.strip('$'),'type':['acne','blemishes'],'url':str(u).split('data-product-master-product-id=')[1][1:9]})
        #for images... for i in images:
            #'image':(str(i).split('"'))[-2]
    df=pd.DataFrame(dfdict)
    return (df,prodlist)

def categorical_columns(colname,lis,dataframe):
    category=[]
    for f in dataframe.prodName:
        if f in lis:
            category.append(1)
        else:
            category.append(0)
    dataframe[colname.replace(',','')]=category

def type_column(dataframe):
    types=['cleanser','exfoliator','makeup-removers','toner','mist','treatment','serum','lotion','moisturizer',
             'balm','oil','mask','peel','lip','eye','supplement','tool']
    typelist=[]
    for prod in dataframe.prodName:
        for t in types:
            if prod in t:
                typelist.append(t)
    return typelist

# def problem_colum(problem,)

In [4]:
cleansers=get_products_by_type('cleansers',14)
exfoliators=get_products_by_type('exfoliators',6)
removers=get_products_by_type('makeup-removers',3)
toners=get_products_by_type('toners',6)
mists=get_products_by_type('mists',3)
treatments=get_products_by_type('treatments',14)
serums=get_products_by_type('serums',16)
lotions=get_products_by_type('lotions',6)
moisturizers=get_products_by_type('moisturizers',23)
balms=get_products_by_type('balms',3)
oils=get_products_by_type('oils',5)
masks=get_products_by_type('masks',9)
peels=get_products_by_type('peels',3)
lips=get_products_by_type('lip-care',4)
eyes=get_products_by_type('eye-care',9)
supplements=get_products_by_type('supplements',1)
tools=get_products_by_type('tools',5)

In [4]:
acnedf,acne = get_products_by_problem('acne-blemishes',10)
agedf,age = get_products_by_problem('anti-aging',30)
darkcdf,darkcircles = get_products_by_problem('dark-circles',5)
drydf,dryness = get_products_by_problem('dry-skin',19)
ncdf,norm= get_products_by_problem('normal-combination',17)
oilydf,oily=get_products_by_problem('oily-skin',17)
sensitivedf,sensi=get_products_by_problem('sensitive-skin',17)
reddf,redness=get_products_by_problem('redness-rosacea',17)
# images=[]
# images=(reddf.image+sensitivedf.image+drydf.image+oilydf.image+ncdf.image+darkcdf.image+agedf.image+acnedf.image)

In [5]:
braa=[]
ratee=[]
pri=[]
ur=[]
for i,r,p,u in zip(acnedf.prodName,acnedf.rating,acnedf.price,acnedf.url):
    braa.append(i)
    ratee.append(r)
    pri.append(p)
    ur.append(u)
for i,r,p,u in zip(agedf.prodName,agedf.rating,agedf.price,agedf.url):
    braa.append(i)
    ratee.append(r)
    pri.append(p)
    ur.append(u)
for i,r,p,u in zip(darkcdf.prodName,darkcdf.rating,darkcdf.price,darkcdf.url):
    braa.append(i)
    ratee.append(r)
    pri.append(p)
    ur.append(u)
for i,r,p,u in zip(drydf.prodName,drydf.rating,drydf.price,drydf.url):
    braa.append(i)
    ratee.append(r)
    pri.append(p)
    ur.append(u)
for i,r,p,u in zip(ncdf.prodName,ncdf.rating,ncdf.price,ncdf.url):
    braa.append(i)
    ratee.append(r)
    pri.append(p)
    ur.append(u)
for i,r,p,u in zip(oilydf.prodName,oilydf.rating,oilydf.price,oilydf.url):
    braa.append(i)
    ratee.append(r)
    pri.append(p)
    ur.append(u)
for i,r,p,u in zip(sensitivedf.prodName,sensitivedf.rating,sensitivedf.price,sensitivedf.url):
    braa.append(i)
    ratee.append(r)
    pri.append(p)
    ur.append(u)
for i,r,p,u in zip(reddf.prodName,reddf.rating,reddf.price,reddf.url):
    braa.append(i)
    ratee.append(r)
    pri.append(p)
    ur.append(u)
datadict=[]
for i,r,p,u in zip(braa,ratee,pri,ur):
    datadict.append({'prodName':i,'rating':r,'price':p,'url':u})
finaldf=pd.DataFrame(datadict)

In [18]:
finaldf.rating=[float(i) for i in finaldf.rating]
finaldf.price=[float(i) for i in finaldf.price]

In [22]:
see=finaldf.set_index('prodName')

## Finalize Full DF

In [29]:
categorical_columns('age',age,finaldf)
categorical_columns('darkcircles',darkcircles,finaldf)
categorical_columns('acne',acne,finaldf)
categorical_columns('dry',dryness,finaldf)
categorical_columns('redness',redness,finaldf)
categorical_columns('sensitive',sensi,finaldf)
categorical_columns('oily',oily,finaldf)
categorical_columns('normal',norm,finaldf)
categorical_columns('cleanser',cleansers,finaldf)
categorical_columns('exfoliator',exfoliators,finaldf)
categorical_columns('makeup-removers',removers,finaldf)
categorical_columns('toner',toners,finaldf)
categorical_columns('mist',mists,finaldf)
categorical_columns('treatment',treatments,finaldf)
categorical_columns('serum',serums,finaldf)
categorical_columns('lotion',lotions,finaldf)
categorical_columns('moisturizer',moisturizers,finaldf)
categorical_columns('balm',balms,finaldf)
categorical_columns('oil',oils,finaldf)
categorical_columns('mask',masks,finaldf)
categorical_columns('peel',peels,finaldf)
categorical_columns('lip',lips,finaldf)
categorical_columns('eye',eyes,finaldf)
categorical_columns('supplement',supplements,finaldf)
categorical_columns('tool',tools,finaldf)

In [129]:
finaldf[finaldf['makeup-removers']==1]

,price,prodName,rating,url,age,darkcircles,acne,dry,redness,sensitive,...,moisturizer,balm,oil,mask,peel,lip,eye,supplement,tool,brand
162,12.0,Daily Bio Cotton Makeup Removers 1.9g,4.33,12281536,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,Dermalogica
890,28.0,Caudalie Instant Foaming Cleanser (5.2oz),5.00,10727688,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,BABOR
947,28.0,Caudalie Gentle Cleansing Milk (7oz),4.67,10727692,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,PIXI
1024,15.0,Sundari Make Up Remover Balm 1.7 oz,5.00,11352183,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,Darphin
1214,28.0,Caudalie Instant Foaming Cleanser (5.2oz),5.00,10727688,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,NaN
1273,28.0,Caudalie Gentle Cleansing Milk (7oz),4.50,10727692,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,NaN
1382,42.0,ESPA Bio-Active Eye Cleanser 100ml,3.75,11553558,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,NaN
1452,20.0,NuFACE Prep-N-Glow Cloths (Pack of 20),5.00,11331872,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
1814,12.0,Daily Bio Cotton Makeup Removers 1.9g,5.00,12281536,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,NaN


In [10]:
with open('CategoricalItemDF.pickle', 'rb') as f:
    finaldf = pickle.load(f)

# Get Description Info & Get Into DF

In [64]:
finaldf.url

0       11174178
1       11289629
2       10915787
3       11174171
4       10364095
5       11289619
6       11679046
7       11561754
8       11976675
9       11561755
10      11289636
11      11289613
12      10420992
13      11314421
14      11561753
15      12031613
16      11872688
17      11175784
18      11561759
19      11888501
20      11561765
21      11561771
22      11561752
23      11292112
24      11289639
25      11289620
26      11561775
27      11289677
28      11561770
29      11289640
          ...   
1854    11833542
1855    11561761
1856    11561768
1886    10892539
1891    11814874
1893    11248383
1894    11174247
1895    11174177
1896    12060145
1897    11833575
1898    11833563
1899    11668592
1902    11289064
1903    11641411
1904    11289709
1905    12060164
1906    11174234
1907    11174172
1908    11588119
1909    11174191
1910    11641412
1911    11107123
1912    11553631
1913    11027650
1914    10922784
1915    10941239
1916    10941234
1917    120565

In [67]:
ingreddict=[]
for (u,p) in zip(finaldf.url,finaldf.prodName):
#     print(u,' ',p)
    response=requests.get('https://www.skinstore.com/{}.html'.format(u))
    soup=BeautifulSoup(response.content,'html.parser')
    info=soup.find('div',{'data-information-component':'ingredients'})
    brand=soup.find('div',{'data-information-component':'brand'})
    try:
        ingreddict.append({'url':u,'brand':brand.text.replace('\n',''),'ingredients':info.text.replace('\n','').split(',')})
    except:
        ingreddict.append({'url':u,'brand':brand.text.replace('\n',''),'ingredients':'NaN'})


In [63]:
info

In [24]:
ingreddf=pd.DataFrame(ingreddict)
# ingreddf.drop_duplicates(inplace=True)

NameError: name 'ingreddict' is not defined

In [29]:
newingred=[]
for i in ingreddf.ingredients:
    newingred.append(i.split(','))

In [76]:
finaldf.drop_duplicates(inplace=True)

In [78]:
ingreddf.drop_duplicates(inplace=True)

TypeError: unhashable type: 'list'

In [95]:
# newfull.reset_index(inplace=True)
# newfull.drop(columns=[index'],inplace=True)
newfull

,brand,ingredients,url_l,price,prodName,rating,url,age,darkcircles,acne,...,lotion,moisturizer,balm,oil,mask,peel,lip,eye,supplement,tool
0,Paula's Choice,"Aqua, Methylpropanediol (hydration), Butylene ...",11174178,25.08,Paula's Choice Skin Perfecting 2% BHA Liquid E...,5.00,11174178,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,SkinCeuticals,"Aqua / Water / Eau, Glycerin, Propylene Glycol...",11289629,105.00,Sunday Riley Good Genes Treatment 30ml,4.73,10915787,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Sunday Riley,"Opuntia Tuna Fruit (Prickly Pear) Extract, Aga...",10915787,62.00,Dermalogica Special Cleansing Gel 16.9oz,4.50,10364095,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Paula's Choice,"Water, Glycerin, Butylene Glycol, Niacinamide ...",11174171,98.00,SkinCeuticals Discoloration Defense Dark Spot ...,5.00,11679046,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Dermalogica,Key Ingredients:Quillaja Saponaria: Bark purif...,10364095,249.00,SkinCeuticals Restore and Hydrate Regimen,5.00,11976675,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,SkinCeuticals,"Aqua/Water/Eau, Phenoxyethanol, Calcium Pantot...",11289619,82.00,SkinCeuticals Retexturing Activator,4.50,11289636,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
12,SkinCeuticals,"Aqua / Water, Butylene Glycol, Niacinamide, Hy...",11679046,70.00,SkinCeuticals Retinol 0.5 Refining Night Treat...,5.00,10420992,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,Slip,100% Silk,11561754,85.00,Slip Silk Pillowcase - Queen - Charcoal,5.00,11561753,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,SkinCeuticals,"Treatment: Aqua/Water, Dipropylene Glycol, Asc...",11976675,31.00,SkinCeuticals MicroExfoliating Cleanser 150ml,5.00,11872688,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18,Slip,100% Silk.,11561755,85.00,Slip Silk Pillowcase - Queen - Silver,4.00,11561759,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [81]:
with open('IngredientsDF.pickle', 'wb') as f:
    pickle.dump(ingreddf, f, pickle.HIGHEST_PROTOCOL)

In [5]:
with open('IngredientsDF.pickle', 'rb') as f:
    idf = pickle.load(f)

In [6]:
idf.head()

,brand,ingredients,url
0,Paula's Choice,"[Aqua, Methylpropanediol (hydration), Butyle...",11174178
1,SkinCeuticals,"[Aqua / Water / Eau, Glycerin, Propylene Gly...",11289629
2,Sunday Riley,"[Opuntia Tuna Fruit (Prickly Pear) Extract, A...",10915787
3,Paula's Choice,"[Water, Glycerin, Butylene Glycol, Niacinam...",11174171
4,Dermalogica,[Key Ingredients:Quillaja Saponaria: Bark puri...,10364095


In [11]:
finaldf['brand']=idf['brand']

In [12]:
finaldf.head()

,price,prodName,rating,url,age,darkcircles,acne,dry,redness,sensitive,...,moisturizer,balm,oil,mask,peel,lip,eye,supplement,tool,brand
0,25.08,Paula's Choice Skin Perfecting 2% BHA Liquid E...,5.00,11174178,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,Paula's Choice
1,67.00,SkinCeuticals Phyto Corrective Gel,4.11,11289629,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,SkinCeuticals
2,105.00,Sunday Riley Good Genes Treatment 30ml,4.73,10915787,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,Sunday Riley
3,17.85,Paula's Choice Skin Balancing Pore-Reducing To...,5.00,11174171,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Paula's Choice
4,62.00,Dermalogica Special Cleansing Gel 16.9oz,4.50,10364095,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Dermalogica


In [15]:
with open('CategoricalItemDF.pickle', 'wb') as f:
    pickle.dump(finaldf, f, pickle.HIGHEST_PROTOCOL)

## Get Review Info & Get Into DF

In [25]:
ratingdict=[]
for u in finaldf.url:
    response=requests.get('https://www.skinstore.com/the-ordinary-aha-30-bha-2-peeling-solution-30ml/{}.html'.format(u))
    soup=BeautifulSoup(response.content,'html.parser')
    titles=soup.findAll('h3',{'class':'productReviews_topReviewTitle'})
    ratings=soup.findAll('div',{'class':'productReviews_topReviewsRatingStarsContainer'})
    contents=soup.findAll('p',{'class':'productReviews_topReviewsExcerpt'})
    dates=soup.findAll('span',{'data-js-element':'createdDate'})
    users=soup.findAll('div',{'class':'productReviews_footerDateAndName'})
    products=soup.find('div',{'data-information-component':'brand'})
    for t,r,c,d,i in zip(titles,ratings,contents,dates,users):
        ratingdict.append({'url':u,'prodName':products,'title':t.text.replace('\n',''),'rating':str(r).split('aria-label=')[1][1:2],'content':c.text.replace('\n','').replace('\r',''),'date':d.text,'user':i.text.replace('\n','').split('by')[1].lower()})
ratingdf=pd.DataFrame(ratingdict)
ratingdf['date']=pd.to_datetime(ratingdf.date)
ratingdf['rating']=[int(r) for r in ratingdf['rating']]
ratingdf.user=ratingdf.user.replace('','user')

In [26]:
# ratingdf.drop_duplicates(inplace=True)
ratingdf.content=[c.lower() for c in ratingdf.content]
ratingdf.shape

(3259, 7)

In [31]:
ratingdf['prodName']=[r.text.replace('\n','') for r in ratingdf['prodName']]

In [32]:
ratingdf

,content,date,prodName,rating,title,url,user
0,this product has changed my problematic skin. ...,2019-08-08,Paula's Choice,5,Amazing,11174178,becky c
1,i absolutely love this product. it's one of th...,2019-05-05,Paula's Choice,5,Amazing,11174178,jess
2,i was really hoping this would work for since ...,2019-04-12,Paula's Choice,5,Awesome,11174178,lisa
3,not sure why everyone doesn't use this amazing...,2018-12-13,Paula's Choice,5,Extraordinary,11174178,stella2
4,i've struggled with breakouts my entire life a...,2018-12-01,Paula's Choice,5,Actually Works!,11174178,carli
5,i finally understand why this has such a cult ...,2018-11-10,Paula's Choice,5,Extraordinary,11174178,stella2
6,amazing product! clears up congested skin but ...,2018-08-24,Paula's Choice,5,Highly recommended!,11174178,tilly
7,"this product is literally a skin saver, that i...",2017-04-04,Paula's Choice,5,MUST HAVE!,11174178,anne206
8,i bought this based on the one and only review...,2015-12-23,Paula's Choice,5,effective,11174178,samsara33139
9,nothing has shrunken my pores like this. and i...,2014-12-28,Paula's Choice,5,Nothing better,11174178,suzi


In [34]:
omgdf=ratingdf[['prodName','rating','url','user']]

In [37]:
omgdf.drop_duplicates(inplace=True)

/Users/mackenziemitchell/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [39]:
omgdf

,prodName,rating,url,user
0,Paula's Choice,5,11174178,becky c
1,Paula's Choice,5,11174178,jess
2,Paula's Choice,5,11174178,lisa
3,Paula's Choice,5,11174178,stella2
4,Paula's Choice,5,11174178,carli
6,Paula's Choice,5,11174178,tilly
7,Paula's Choice,5,11174178,anne206
8,Paula's Choice,5,11174178,samsara33139
9,Paula's Choice,5,11174178,suzi
10,SkinCeuticals,5,11289629,kiko


In [38]:
with open('recdf.pickle', 'wb') as f:
    pickle.dump(omgdf, f, pickle.HIGHEST_PROTOCOL)

In [33]:
with open('userratingsDF.pickle', 'wb') as f:
    pickle.dump(ratingdf, f, pickle.HIGHEST_PROTOCOL)